In [20]:
import pandas as pd
file_1 = pd.read_excel('annotation_(framelevel).xlsx')
file_2 = pd.read_excel('annotation(sound_time unit).xlsx')

#image_text
video_df = pd.DataFrame(file_1)  
#sound
sound_df = pd.DataFrame(file_2)

In [21]:
sound_df.columns

Index(['Video ID', 'Title', 'Filename', 'Violence(Sound) Type',
       'Duplicated moment with previous', 'Sound Start', 'Start time(s)',
       'Sound End', 'End time(s)', 'Start frame', 'End frame', 'Sound Type',
       'Memos', 'Max frame', 'Max Time'],
      dtype='object')

In [22]:
video_df.columns

Index(['Video ID', 'Filename', 'Start Frame', 'End Frame', 'Start Time (s)',
       'End Time (s)', 'Violence Type (Video)', 'Video Type',
       'Violence Type (Sound)', 'Sound Type', 'Violence Type (Text)',
       'Manual Text', 'Memo', 'Max Frames', 'Max time'],
      dtype='object')

In [32]:
sound_df = sound_df.rename(columns={
    'Start frame' : 'sound_start_frame',
    'End frame' : 'sound_end_frame',
    'Sound Type' : 'sound_label'
    })

In [33]:
sound_df.columns

Index(['Video ID', 'Title', 'Filename', 'Violence(Sound) Type',
       'Duplicated moment with previous', 'Sound Start', 'Start time(s)',
       'Sound End', 'End time(s)', 'sound_start_frame', 'sound_end_frame',
       'sound_label', 'Memos', 'Max frame', 'Max Time'],
      dtype='object')

In [34]:
sound_df_refined = sound_df.drop(columns =[
                                 'Filename',
                                 'Duplicated moment with previous',
                                 'Sound Start',
                                 'Sound End',
                                 'Max frame',
                                 'Max Time'])
sound_df_refined

,Video ID,Title,Violence(Sound) Type,Start time(s),End time(s),sound_start_frame,sound_end_frame,sound_label,Memos
0,100,Teen girls captured on video in nasty parking ...,Violent,14.00,17.00,421.000000,519.000000,Screaming,NaN
1,100,Teen girls captured on video in nasty parking ...,Violent,20.00,23.00,599.428740,689.343051,Screaming,NaN
2,100,Teen girls captured on video in nasty parking ...,Violent,24.00,37.00,719.314488,1108.943170,Screaming,NaN
3,100,Teen girls captured on video in nasty parking ...,Violent,43.00,53.00,1288.771792,1588.486162,Screaming,NaN
4,107,Delivery Driver Saves Homeowner & Dog from Pit...,Violent,44.00,47.00,1342.000000,1394.000000,Screaming,NaN
...,...,...,...,...,...,...,...,...,...
235,24,RAW VIDEO Wild shootout in Northeast DC captur...,Violent,45.93,47.00,689.000000,705.000000,Gunshot,NaN
236,24,RAW VIDEO Wild shootout in Northeast DC captur...,Violent,49.00,56.07,735.000000,841.000000,Gunshot,NaN
237,24,RAW VIDEO Wild shootout in Northeast DC captur...,Violent,60.00,64.00,900.000000,960.000000,Others,Siren
238,24,RAW VIDEO Wild shootout in Northeast DC captur...,Violent,64.07,70.07,961.000000,1051.000000,Shouting,NaN


In [35]:
video_df = video_df.rename(columns={
    'Start Frame' : 'Start frame',
    'End Frame' : 'End frame',
    'Violence Type (Sound)' : 'Violence(Sound) Type',
    'Start Time (s)' : 'Start time(s)',
    'End Time (s)' : 'End time(s)'
})

In [37]:
video_df.columns

Index(['Video ID', 'Filename', 'Start frame', 'End frame', 'Start time(s)',
       'End time(s)', 'Violence Type (Video)', 'Video Type',
       'Violence(Sound) Type', 'Sound Type', 'Violence Type (Text)',
       'Manual Text', 'Memo', 'Max Frames', 'Max time'],
      dtype='object')

In [52]:
video_df_refined = video_df.astype({'Video ID': int, 'Start frame':int, 'End frame':int})
sound_df_refined = sound_df_refined.astype({'Video ID':int, 'sound_start_frame':int, 'sound_end_frame':int})

In [57]:
def extract_sound_columns(row, max_sound = 3):
    matches = sound_df_refined[
        (sound_df_refined['Video ID'] == row['Video ID']) &
        (sound_df_refined['sound_start_frame'] >= row['Start frame']) &
        (sound_df_refined['sound_end_frame'] <= row['End frame'])
    ]
    sound_cols = {}
    for idx, (_, match) in enumerate(matches.iterrows()):
        if idx >= max_sound:
            print('More than 3 sound matches found, skipping additional matches.')
            break
        i = idx + 1
        sound_cols[f'sound_{i}'] = match['sound_label']
        sound_cols[f'sound_{i}_start_frame'] = match['sound_start_frame']
        sound_cols[f'sound_{i}_end_frame'] = match['sound_end_frame']
    
    # Fill in missing ones with None
    current_count = len(matches)
    for i in range(current_count, max_sound+1):
        sound_cols[f'sound_{i}'] = None
        sound_cols[f'sound_{i}_start_frame'] = None
        sound_cols[f'sound_{i}_end_frame'] = None
    
    return pd.Series(sound_cols)

In [81]:
def extract_sound_columns(row, sound_df, max_sounds=3):
    video_id = row['Video ID']
    start_frame = row['Start frame']
    end_frame = row['End frame']
    
    # Filter matching sound annotations
    matches = sound_df[
        (sound_df['Video ID'] == video_id) &
        (sound_df['sound_start_frame'] >= start_frame) &
        (sound_df['sound_end_frame'] <= end_frame)
    ].sort_values('sound_start_frame')

    # Prepare output columns dynamically
    sound_cols = {}
    for idx, match in enumerate(matches.itertuples(), start=1):
        if idx > max_sounds:
            break
        sound_cols[f'sound_{idx}'] = match._8   # Assuming 8th column is 'Sound Ty'
        sound_cols[f'sound_{idx}_start_frame'] = match._6  # 'Start frame'
        sound_cols[f'sound_{idx}_end_frame'] = match._7    # 'End frame'


    for idx in range(len(matches) + 1, max_sounds + 1):
        sound_cols[f'sound_{idx}'] = None
        sound_cols[f'sound_{idx}_start_frame'] = None
        sound_cols[f'sound_{idx}_end_frame'] = None

    return pd.Series(sound_cols)


In [83]:
sounds_cols_df = video_df_refined.apply(lambda row: extract_sound_columns(row, sound_df, max_sounds=3), axis=1)
video_df_refined = pd.concat([video_df_refined, sounds_cols_df], axis=1)

video_df_refined.to_csv('video_df_refined.csv', index=False)